In [1]:
#load the grouped list for colnames
from utils import *
#目前只有mary road有tc,我们在做union model时候先不用tc
#可以单独给mary road做一个individual模型，里面使用tc
weathers_2.remove('tc')

# air_pollutions, weathers, weathers_2, times
####
# continuous_variables, category_variables, percentile_variables

In [2]:
import json
import numpy as np
import pandas as pd
import os
import random
from datetime import datetime

In [3]:
data_folder = 'D:/Work/污染项目/data-overall-update/' \
              'data_process/before_engineer/union_per_pollution/'

output_folder = 'D:/Work/污染项目/data-overall-update/' \
              'data_process/after_engineer/union_per_pollution/'

In [4]:
air_pollutions, weathers, weathers_2, times

(['nox', 'no2', 'no', 'o3', 'pm2.5', 'pm10'],
 ['ws', 'wd', 'temp', 'RH'],
 ['ssr', 'tp', 'blh', 'tcc', 'sp'],
 ['date_unix', 'week', 'weekday', 'hour', 'month', 'day_julian'])

In [5]:
continuous_variables, category_variables, percentile_variables

(['ws',
  'wd',
  'temp',
  'RH',
  'ssr',
  'tp',
  'blh',
  'tcc',
  'sp',
  'ap_lat',
  'ap_long',
  'met_lat',
  'met_long',
  'date_unix',
  'year'],
 ['ap_code',
  'MetCode',
  'site',
  'week',
  'weekday',
  'hour',
  'month',
  'day_julian'],
 ['ws', 'wd', 'temp', 'RH', 'ssr', 'tp', 'blh', 'tcc', 'sp'])

In [6]:
def isnull_ratio(df, target_colname):
    null_cnts =  df[df[target_colname].isnull()].shape[0]
    return null_cnts/df.shape[0]

# from date unix extract year
def extract_year(row):
    date_unix = int(row['date_unix'])
    return datetime.utcfromtimestamp(date_unix).year

def fill_continuous_missValues(air_pollution,
                               data_folder,
                               n_fold):

    df = pd.read_csv(data_folder+air_pollution+'_unionSites.csv')
    
    #add column year
    df['year'] = df.apply(lambda row : extract_year(row), axis=1)

    #the sites related for this pollution
    related_sites = df.ap_code.unique()
    
    #missing values的行，直接用那一列在那个站点的中位数代替就可以
    no_missing_dfs = []

    for site in related_sites:

        df_site = df[df['ap_code']==site]

        for colname in continuous_variables:
            df_site[colname] = df_site[colname].fillna(df_site[colname].median())
        
        #ids for cross validation at each site level
        ids = list(range(df_site.shape[0]))
        random.shuffle(ids)
        ids = [item % n_fold for item in ids]
        df_site['cv_fold'] = ids

        no_missing_dfs.append(df_site)

    df = pd.concat(no_missing_dfs)
    return df

def percentiles_return(df, percentile_variables):
    
    percentiles = [25, 50, 75]
    row_cnt = df.shape[0]
    df_percentile = []
    df_percentile_columns = []
    
    for colname in percentile_variables:
        for percentile in percentiles:
            df_percentile.append([np.percentile(df[colname], percentile)]*row_cnt)
            df_percentile_columns.append(colname+'_percentile'+str(percentile))
            
    return pd.DataFrame(df_percentile, columns=df_percentile_columns)
        
    

#get and save all percentiles at ap_code level into a dictionary
def getNsave_percentiles_dict(df, 
                              percentile_variables, 
                              air_pollution, 
                              output_folder):
    #save all percentiles at ap_code level into a dictionary

    percentiles_dict = {}
    percentiles = [25, 50, 75]

    for colname in percentile_variables:
        percentiles_dict[colname]={}
        for ap_code in df.ap_code.unique():
            df_code = df[df['ap_code']==ap_code]
            percentiles_dict[colname][ap_code] = {
                '25': np.percentile(df_code[colname], 25),
                '50': np.percentile(df_code[colname], 50),
                '75': np.percentile(df_code[colname], 75)
            }



    # Serializing json
    json_object = json.dumps(percentiles_dict, indent=4)

    # Writing to sample.json
    with open(output_folder+air_pollution+"_percentiles_dict_ap_code_lvl.json", "w") as outfile:
        outfile.write(json_object)
    
    return percentiles_dict


# get the percentile for specific column at ap_code level
def col_percentile(row, colname, percentile, percentiles_dict):
    ap_code = row['ap_code']
    return percentiles_dict[colname][ap_code][percentile]

# 1. 填补missing values用median填补
# 2. 加上cross validation column
# 3. 加上新的features: get and save all percentiles for each column at 25,50,75 at ap_code level(这个先不用)

In [10]:
for air_pollution in air_pollutions:
    #fillin missing values and add cross validation column
    df = fill_continuous_missValues(air_pollution,
                                    data_folder,
                                    n_fold = 5)
    

#     percentiles_dict = getNsave_percentiles_dict(df, 
#                                   percentile_variables, 
#                                   air_pollution, 
#                                   "percentiles/")


#     #col_percentile(row, colname, percentile, percentiles_dict):

#     for colname in percentile_variables:
#         for percentile in ['25', '50', '75']:
#             df[colname+'_'+percentile] = df.apply(lambda row : col_percentile(row, 
#                                                                               colname, 
#                                                                               percentile, 
#                                                                               percentiles_dict), axis=1)


        
    df.to_csv(output_folder+air_pollution+'_unionSites.csv', index=False)

C:\Users\rxt29\AppData\Local\Temp/ipykernel_14412/4221650816.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_site[colname] = df_site[colname].fillna(df_site[colname].median())
C:\Users\rxt29\AppData\Local\Temp/ipykernel_14412/4221650816.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_site['cv_fold'] = ids
